In [1]:
from sklearn.base import BaseEstimator
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data.csv',sep=';')

In [3]:
wEducation = 0.05
wExperience = 0.05
wIndustry = 0.2
wInterest = 0.1
wLocation = 0.2
wLookin4 = 0.2
wOffer = 0.2
wSkill = 0.1

In [4]:
df.fillna(-1,inplace=True)

In [5]:
def compare2Lists(l1,l2):
    if l1 ==-1 or l2 == -1:
        return 0
    else:
        lens = len(set(str(l1).split(',')) & set(str(l2).split(',')))
        return lens

In [6]:
class Algo(BaseEstimator):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.X = X
    
    def predict(self, X):
        self.X['score'] = 0
        for index, row in self.X.iterrows():
            #print(str(row.education))
            if compare2Lists(row.education,X.education) > 0:
                self.X.loc[index,'score']+=wEducation
            if compare2Lists(row.experience,X.experience) > 0:
                self.X.loc[index,'score']+=wExperience
            if compare2Lists(row.industry,X.industry) > 0:
                self.X.loc[index,'score']+=wIndustry
            if compare2Lists(row.interest,X.interest) > 0:
                self.X.loc[index,'score']+=wInterest
            if compare2Lists(row.location,X.location) > 0:
                self.X.loc[index,'score']+=wLocation
            if compare2Lists(row.lookin4,X.offer) > 0:
                self.X.loc[index,'score']+=wLookin4
            if compare2Lists(row.offer,X.lookin4) > 0:
                self.X.loc[index,'score']+=wOffer
            if compare2Lists(row.skill,X.skill) > 0:
                self.X.loc[index,'score']+=wSkill
        return self.X

In [7]:
algo = Algo()

In [8]:
algo.fit(df)

In [49]:
df2 = algo.predict(df.loc[10])

In [50]:
df2

,education,experience,industry,interest,location,lookin4,offer,skill,uid,score
0,-1,-1,"19,111,53,127",-1,"Geneva Area, Switzerland","A Mentor,Funding,Co-founders","Co-founders,Investment",-1,59f70ab82706222d006f28ca,0.0
1,-1,-1,"125,94,111",-1,"Paris Area, France","A Mentor,Co-founders,Industry Expertise","Business Partner,Consultancy Work,Internship",-1,59f70abc2706222d006f28cb,0.2
2,-1,-1,"94,50",-1,"Paris Area, France","Clients,Funding,Internship","Exchange of Ideas,Industry Expertise,Investmen...",-1,59f70abd2706222d006f28cc,0.2
3,-1,-1,-1,-1,Sunnyvale,A New Job,-1,-1,59f70abf2706222d006f28cd,0.0
4,-1,Lector,"68,70,103",Photography,Strasbourg,"A New Job,A Mentor,Exchange of Ideas","Consultancy Work,Exchange of Ideas,Freelancing",Funny,59f70ac02706222d006f28ce,0.6
5,-1,-1,"140,44",-1,Carentan,"A New Job,A Mentor,Clients,Exchange of Ideas,I...","Exchange of Ideas,Mentorship,Professional Netw...",-1,59f70ac42706222d006f28cf,0.4
6,-1,-1,"122,10,105,144,33,96",-1,"Paris Area, France","Exchange of Ideas,A Mentor,I am hiring","Consultancy Work,Job Positions,Exchange of Ideas",-1,59f70ac62706222d006f28d0,0.6
7,-1,-1,"127,94,53,50",-1,"Paris Area, France","Co-founders,A Mentor,Funding,Internship","Exchange of Ideas,Internship,Investment Opport...",-1,59f70ac82706222d006f28d1,0.2
8,Bioinformatics,"Co-founder,Co-founder & CEO","12,6,96,4,15","Entrepreneurship,Outdoor Activities,Playing Mu...",Paris,"Business Partner,Co-founders,Exchange of Ideas...","Consultancy Work,Industry Expertise,Services/P...","Entrepreneurship,Innovation management,Present...",59f70acb2706222d006f28d3,0.4
9,-1,-1,"6,5,4,3,118",-1,"Paris Area, France","Business Partner,Clients,Co-founders,Exchange ...","Business Partner,Co-founders,Freelancing,Excha...",-1,59f70acd2706222d006f28d4,0.6


In [ ]:
df2 = df2.sort_values(by=['score'],ascending=False).reset_index(drop=True)

In [ ]:
df2.head(200)

In [ ]:
len(df2.values)

In [9]:
results = []

In [10]:
uids = df.uid.values

In [11]:
for i in range(uids.size):
    results.append(algo.predict(df.loc[i]).score.values)

In [32]:
df3 = pd.DataFrame(data=results, index=uids,dtype=np.float64, columns=uids)

In [37]:
df4 = pd.read_csv('data2.csv',sep=';',index_col='Unnamed: 0')

In [14]:
df5 = df3*df4

In [18]:
score = df5.values.sum()/(uids.size*uids.size)

In [19]:
score

0.003110712557912493

In [42]:
import numpy as np  
from sklearn.preprocessing import LabelBinarizer  
from sklearn.metrics import make_scorer  
from sklearn.utils import check_X_y  
import sys  
  
def dcg_score(y_true, y_score, k=5):  
    order = np.argsort(y_score)[::-1]  
    y_true = np.take(y_true, order[:k])  
    gain = 2 ** y_true - 1  
    #print(gain)  
    discounts = np.log2(np.arange(len(y_true)) + 2)
    #print(discounts)  
    return np.sum(gain / discounts)  
  
def ndcg_score(y_true, y_score, k=5):  
    #y_score, y_true = check_X_y(y_score, y_true)  
  
    # Make sure we use all the labels (max between the length and the higher  
    # number in the array)  
    ##lb = LabelBinarizer()  
    ##lb.fit(np.arange(max(np.max(y_true) + 1, len(y_true))))  
    ##binarized_y_true = lb.transform(y_true)  
    binarized_y_true = y_true
    #y_score = np.array(y_score)
    #print(binarized_y_true)  
    if binarized_y_true.shape != y_score.shape:
        print(binarized_y_true.shape)
        print(y_score.shape)
        raise ValueError("y_true and y_score have different value ranges")  
  
    scores = []  
  
    # Iterate over each y_value_true and compute the DCG score  
    for y_value_true, y_value_score in zip(binarized_y_true, y_score):  
        actual = dcg_score(y_value_true, y_value_score, k)  
        best = dcg_score(y_value_true, y_value_true, k)
        #print(best)  
        scores.append(actual / best)  
    return np.mean(scores)  

In [47]:
print(ndcg_score(df4.values,df3.values,k=100))

0.21124200435573476


In [51]:
df3

,59f70ab82706222d006f28ca,59f70abc2706222d006f28cb,59f70abd2706222d006f28cc,59f70abf2706222d006f28cd,59f70ac02706222d006f28ce,59f70ac42706222d006f28cf,59f70ac62706222d006f28d0,59f70ac82706222d006f28d1,59f70acb2706222d006f28d3,59f70acd2706222d006f28d4,...,5ac6918f4176e5001fa1d6d3,5ac698424176e5001fa1d6fd,5ac70d704176e5001fa1d820,5ac773fc4176e5001fa1d8c2,5ac7b06b4176e5001fa1d92e,5ac855bb4176e5001fa1d988,5aca14174176e5001fa1db56,5acab40a4176e5001fa1dc13,5acacbb14176e5001fa1dc2f,5acad3f14176e5001fa1dc31
59f70ab82706222d006f28ca,0.8,0.4,0.0,0.0,0.00,0.0,0.0,0.4,0.2,0.4,...,0.0,0.0,0.0,0.2,0.0,0.0,0.4,0.0,0.0,0.2
59f70abc2706222d006f28cb,0.4,0.4,0.8,0.0,0.00,0.0,0.2,0.6,0.4,0.6,...,0.2,0.0,0.2,0.0,0.2,0.2,0.4,0.0,0.2,0.4
59f70abd2706222d006f28cc,0.0,0.8,0.4,0.0,0.20,0.2,0.4,0.6,0.2,0.4,...,0.2,0.2,0.2,0.2,0.2,0.4,0.0,0.2,0.2,0.4
59f70abf2706222d006f28cd,0.0,0.0,0.0,0.2,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59f70ac02706222d006f28ce,0.0,0.0,0.2,0.0,1.05,0.4,0.4,0.2,0.2,0.4,...,0.4,0.2,0.2,0.4,0.2,0.6,0.0,0.2,0.0,0.2
59f70ac42706222d006f28cf,0.0,0.0,0.2,0.0,0.40,0.8,0.4,0.2,0.2,0.4,...,0.4,0.4,0.0,0.4,0.2,0.4,0.0,0.2,0.2,0.4
59f70ac62706222d006f28d0,0.0,0.2,0.4,0.0,0.40,0.4,0.8,0.4,0.4,0.6,...,0.2,0.4,0.2,0.4,0.4,0.6,0.0,0.2,0.0,0.2
59f70ac82706222d006f28d1,0.4,0.6,0.6,0.0,0.20,0.2,0.4,0.8,0.2,0.6,...,0.2,0.2,0.2,0.2,0.2,0.4,0.2,0.2,0.2,0.2
59f70acb2706222d006f28d3,0.2,0.4,0.2,0.0,0.20,0.2,0.4,0.2,0.7,0.4,...,0.6,0.2,0.0,0.6,0.2,0.6,0.4,0.0,0.0,0.4
59f70acd2706222d006f28d4,0.4,0.6,0.4,0.0,0.40,0.4,0.6,0.6,0.4,0.8,...,0.2,0.4,0.2,0.6,0.4,0.6,0.4,0.2,0.0,0.2
